In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import scipy
import os
import pickle
import librosa
import librosa.display
import IPython.display as ipd
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential

# Paths
METADATA_PATH = "../reports/1_Explore_Metadata/valid_track_genres.csv"
AUDIO_DIR = "../data/fma_small/"
FEATURE_SAVE_PATH = "../features/knn_features.npz"
MODEL_SAVE_PATH = "../models/knn_model.pkl"
TEST_OUTPUTS_PATH = "../features/knn_test_outputs.npz"
REPORT_DIR = "../reports/4_KNN_Classifier/"
os.makedirs(REPORT_DIR, exist_ok=True)

def load_metadata(path):
    df = pd.read_csv(path)
    df = df.dropna(subset=['genre_top'])
    return df


ModuleNotFoundError: No module named 'pandas'

In [ ]:
audio ="../data/songs/Classic_Rock/001. Eagles - Hotel California (2013 Remaster).mp3"
data,sr=librosa.load(audio)
print(type(data),type(sr))

librosa.load(audio,sr=45600)

y = librosa.stft(data)  
S_db = librosa.amplitude_to_db(np.abs(y), ref=np.max)

import IPython
IPython.display.Audio(data,rate=sr)

In [ ]:
plt.figure(figsize=(7,4))
librosa.display.waveshow(data,color="#2B4F72", alpha = 0.5)
plt.show()

In [ ]:
# Spectrogram of the audio
stft=librosa.stft(data)
stft_db=librosa.amplitude_to_db(abs(stft))
plt.figure(figsize=(7,6))
librosa.display.specshow(stft_db,sr=sr,x_axis='time',y_axis='hz')
plt.colorbar()

In [ ]:
data, sr = librosa.load(audio)
plt.figure(figsize=(7, 3))
librosa.display.waveshow(data, sr=sr,alpha=0.4)
plt.title('Waveplot - ROCK')

# Creating log mel spectrogram
plt.figure(figsize=(7, 4))
spectrogram = librosa.feature.melspectrogram(y=data, sr=sr, n_mels=128,fmax=8000) 
spectrogram = librosa.power_to_db(spectrogram)
librosa.display.specshow(spectrogram, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram - ROCK')
plt.colorbar(format='%+2.0f dB');

# Playing audio
ipd.Audio(audio)
print("Columns containing missing values",list(df.columns[df.isnull().any()]))
print(df.iloc[:,:-1])

In [ ]:
df=df.drop(labels="filename",axis=1)

from sklearn.preprocessing import StandardScaler
fit=StandardScaler()
X=fit.fit_transform(np.array(df.iloc[:,:-1],dtype=float))

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)


In [ ]:
# Applying K nearest Neighbour algorithm to predict the results
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

clf1=KNeighborsClassifier(n_neighbors=3)
clf1.fit(X_train,y_train)
y_pred=clf1.predict(X_test)
print("Training set score: {:.3f}".format(clf1.score(X_train, y_train)))
print("Test set score: {:.3f}".format(clf1.score(X_test, y_test)))
cf_matrix = confusion_matrix(y_test, y_pred)
sns.set(rc = {'figure.figsize':(8,3)})
sns.heatmap(cf_matrix, annot=True)
print(classification_report(y_test,y_pred))